In [1]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine
import numpy as np
import dedupe
import re
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
PG_CONNECT = os.getenv("PG_CONNECT")

In [3]:
# Residential Land Use Codes from MA Dept of Revenue
# https://www.mass.gov/files/documents/2016/08/wr/classificationcodebook.pdf
# Codes are 101*-109*, 031*, and 013*
# Often include suffixes (letters, zeroes or no character), thus regex *?
USE_CODES = '^1[0-1][1-9]*?|^013*?|^031*?'
BOS_CODES = '^R[1-4]$|^RC$|^RL$|^CD$|^A$'

medparse = lambda x: pd.datetime.strptime(x, '%Y%m%d')

def read_res(file_dict):
    df = pd.DataFrame()
    for town, file, in file_dict.items():
        town_df = gpd.read_file(file).drop('geometry', axis='columns')
        town_df['town'] = town
        df = df.append(town_df, ignore_index=True)
    return df

In [4]:
# Data from MassGIS Standardized Assessor's Parcels
# https://docs.digital.mass.gov/dataset/massgis-data-standardized-assessors-parcels
# Medford, Cambridge, and Somerville all last updated FY 2019

In [5]:
# Medford processing.
files = {'med': './data/assess/med_assess.dbf'}
med = read_res(files)
# Rename column to lower-case.
med.columns = med.columns.str.lower()
# Filter for residential paWorldrcels.
med = med[med['use_code'].str.contains(USE_CODES, regex=True)]
# Identify rows with co-owner names erroneously listed in address column.
mask = med.own_addr.str.contains(pat = '|'.join(['^C/O', '^[A-Za-z]']), na=False) & ~med.own_addr.str.contains(pat = '|'.join(['^PO', '^P.O.', '^P. O.', '^P O ', '^ONE', '^BOX', '^ZERO']), na=False)
# Add co-owners identified to co column.
med['co'] = med.own_addr[mask]
med.loc[~mask, 'co'] = None
# Fill own_addr with none for above-identified rows.
med.loc[mask, 'own_addr'] = None
# Remame columns
med = med.rename(columns = {
    'prop_id': 'gisid',
    'owner1': 'own',
    'site_addr': 'prop_addr',
    'total_val': 'ass_val',
    'location': 'unit',
    'ls_date': 'sale_d',
    'ls_price': 'sale_p'
})
med.loc[:,'sale_d'] = pd.to_datetime(med['sale_d'], format='%Y%m%d')
med.loc[:,'prop_addr'] = med.prop_addr.str.strip()
# Replace underscores with hyphens.
med.loc[:,'gisid'] = med.gisid.str.replace(r'_', '-', regex=True)
# Concatenate address.
med.loc[:,'own_addr'] = [', '.join((str(a), str(b), str(c))) for a, b, c in zip(med['own_addr'], med['own_city'], med['own_state'])]
med.loc[:,'own_addr'] = [' '.join((str(a), str(b))) for a, b in zip(med['own_addr'], med['own_zip'])]
# Remove concatenated Nones.
med = med.replace({r'None, ': ''}, regex=True)
med['year'] = 'FY2019'
med.loc[:,'sale_p'] = med['sale_p'].replace(0, None)
# Filter columns.
med = med[['gisid', 'town', 'prop_addr', 'unit', 'own', 'co', 'own_addr', 'ass_val', 'year', 'sale_d', 'sale_p']]

In [6]:
# Somerville processing.
som = pd.read_csv('./data/assess/som_assess_FY14-FY19.csv',
                  dtype={'HOUSE NO': str}) 
som.columns = som.columns.str.lower()

# Filter for residential parcels.
som = som[som['pcc'].str.contains(USE_CODES, regex=True)]

som.loc[:,'prop_addr'] = [' '.join((str(a), str(b))) for a, b in zip(som['house no'], som['street'])]

som = som.replace({r'^, ': '', r' ,': '', r', nan': '', r'nan': '', r'None, ': '', r', None': ''}, regex=True)
som = som.replace({' ': None, '': None, np.nan: None})

# Pad ZIP code with zeroes, remove 4-digit suffix.
# Assessor appears to have overzealously corrected...
som.loc[:,'owner zip'] = som['owner zip'].str[1:]
som.loc[:,'own_addr'] = [', '.join((str(a), str(b), str(c), str(d))) for a, b, c, d in zip(som['owner add'], som['owner city'], som['owner state'], som['owner zip'])]

som.loc[:,'gisid'] = ['-'.join((str(m), str(b), str(l))) for m, b, l in zip(som['map'], som['block'], som['lot'])]
som['town'] = 'som'
som = som.drop(['year'], axis=1)
som = som.rename(columns = {
    'commitment owner': 'own',
    'current co-owner': 'co',
    'parcel val': 'ass_val',
    'fiscal_year': 'year'
})

# Assessor seems to have screwed up this column in the 2014-2019 data
# but it appears that 2019 data is incrementally numbered (¯\_(ツ)_/¯)
som = som.loc[som['year'] >= 2019]
som['year'] = 'FY2019'
# Filter columns.
som = som[['gisid', 'town', 'prop_addr', 'unit', 'own', 'co', 'own_addr', 'ass_val', 'year']]

/home/ericrobskyhuntley/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (13,20,53,54,59,60,61,62,63,64,65,66,67,68,69,70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
# Somerville 2019 assessor's table doesn't include sale date 
# (apparently by accident), so we collate with MassGIS source.
files = {'som': './data/assess/som_massgis.dbf'}
som_mg = read_res(files)
# Rename column to lower-case.
som_mg.columns = som_mg.columns.str.lower()
# Remame columns
som_mg = som_mg.rename(columns = {
    'prop_id': 'gisid',
    'ls_date': 'sale_d',
    'ls_price': 'sale_p'
})
som_mg.loc[:,'sale_d'] = pd.to_datetime(som_mg['sale_d'], format='%Y%m%d')
# Replace underscores with hyphens.
som_mg.loc[:,'gisid'] = som_mg.gisid.str.replace(r'_', '-', regex=True)
som_mg.loc[:,'sale_p'] = som_mg['sale_p'].replace(0, None)
# Filter columns.
som_mg = som_mg[['gisid', 'sale_d', 'sale_p']]

In [8]:
som = som.merge(som_mg[['gisid', 'sale_d', 'sale_p']], how='left', on='gisid')

In [73]:
bos = pd.read_csv('./data/assess/bos_assess.csv', dtype={'GIS_ID': str, 'MAIL_ZIPCODE': str, 'U_TOT_RMS': str})
bos.columns = bos.columns.str.lower()
bos = bos.rename(columns = {
    'gis_id': 'gisid',
    'owner': 'own',
    'mail_addressee': 'co',
    'unit_num': 'unit',
    'av_total': 'ass_val'
})
bos['town'] = 'bos'
# Filter by residential property types.
bos = bos[bos['lu'].str.contains(BOS_CODES, regex=True)]
bos.loc[:, 'gisid'] = bos.gisid.str.strip().str.pad(width=10, side='left', fillchar='0')
# Pad ZIP code with zeroes, remove 4-digit suffix.
bos.loc[:,'mail_zipcode'] = bos.mail_zipcode.astype(str).str.strip().str.pad(width=5, side='left', fillchar='0')
# Add comma between city and state.
bos.loc[:,'mail cs'] = bos['mail cs'].str.rsplit(' ', 1).apply(lambda x: ', '.join(x))
# Concatenate property address components
bos.loc[:,'prop_addr'] = [' '.join((str(a), str(b), str(c))) for a, b, c in zip(bos['st_num'], bos['st_name'], bos['st_name_suf'])]
bos.loc[:,'prop_addr'] = bos.prop_addr.str.strip()
# Concatenate owner address components.
bos.loc[:,'own_addr'] = [', '.join((str(a), str(b))) for a, b in zip(bos['mail_address'], bos['mail cs'])]
bos.loc[:,'own_addr'] = [' '.join((str(a), str(b))) for a, b in zip(bos['own_addr'], bos['mail_zipcode'])]
bos.loc[:,'own_addr'] = bos.own_addr.str.strip()
# Filter columns
bos['year'] = 'FY2020'
bos = bos[['gisid', 'town', 'prop_addr', 'unit', 'own', 'co', 'own_addr', 'ass_val', 'year']]
# Replace blank strings with None (necessary for dedupe).
bos = bos.replace({' ': None, '': None, r' #nan': None})
bos = bos.replace({r' #nan': ''}, regex=True)

In [10]:
cam = pd.read_csv('./data/assess/cam_assess.csv',
                  parse_dates=['SaleDate'],
                  dtype={'Owner_Zip': str, 
                         'SalePrice': float,
                         'StateClassCode': str
                        })
# rename all columns to lowercase
cam.columns = cam.columns.str.lower()
# Filter for residential properties.
cam = cam[cam['stateclasscode'].str.contains(USE_CODES, regex=True)]
# Pad zip to five digits and remove 4-digit zip suffix.
cam.loc[:,'owner_zip'] = cam['owner_zip'].str.rsplit('-', 1).str[0]
# Identify rows with co-owner names erroneously listed in address column.
mask = cam.owner_address.str.contains(pat = '|'.join(['^C/O', '^ATTN:']), na=False)
cam.loc[mask, 'owner_address'] = None
# Add co-owners identified to co column.
cam.loc[mask, 'owner_coownername'] = [', '.join((str(a), str(b)))  for a, b in zip(cam.loc[mask, 'owner_coownername'], cam.loc[mask, 'owner_address'])]
# Concatenate owner address components
cam.loc[:,'own_addr'] = [', '.join((str(a), str(b), str(c), str(d))) for a, b, c, d in zip(cam['owner_address'], cam['owner_address2'], cam['owner_city'], cam['owner_state'])]
cam.loc[:,'own_addr'] = [' '.join((str(a), str(b))) for a, b in zip(cam['own_addr'], cam['owner_zip'])]
cam.loc[:,'own_addr'] = cam.own_addr.str.strip()
# Clean property address column
cam['prop_addr'] = cam['address'].str.rsplit('\nCambridge, MA', 1).apply(lambda x: x[0].replace('\n', ' ').strip())
# Bring property address in line with others.
cam['town'] = 'cam' 
cam = cam.rename(columns = {
    'owner_name': 'own',
    'owner_coownername': 'co',
    'assessedvalue': 'ass_val',
    'saleprice': 'sale_p',
    'saledate': 'sale_d'
})
cam['year'] = 'FY2020'
cam['sale_p'].values[cam['sale_p'].values < 1] = None
cam = cam.replace({r'^, ': '', r' ,': '', r', nan': '', r'None, ': '', r', None': ''}, regex=True)
cam = cam.replace({' ': None, '': None, np.nan: None})
cam = cam[['gisid', 'town', 'prop_addr', 'unit', 'own', 'co', 'own_addr', 'ass_val', 'year', 'sale_d', 'sale_p']]

In [11]:
brook = pd.read_csv('./data/assess/brook_assess.csv', 
                    dtype={'SALEPRICE': float,
                          'USECD': str},
                    parse_dates=['SALEDATE'])
brook.columns = brook.columns.str.lower()
brook = brook[brook['usecd'].str.contains(USE_CODES, regex=True)]

brook.loc[:,'zip'] = brook['zip'].str.rsplit('-', 1).str[0]
# Name town.
brook['town'] = 'brk' 
# Concatenate address.
brook.loc[:,'own_addr'] = [', '.join((str(a), str(b), str(c))) for a, b, c, in zip(brook['address'], brook['city'], brook['state'])]
# Append zip to address with no comma.
brook.loc[:,'own_addr'] = [' '.join((str(a), str(b))) for a, b in zip(brook['own_addr'], brook['zip'])]
brook.loc[:,'own_addr'] = brook.own_addr.str.strip()
# Concatenate property address components
brook.loc[:,'prop_addr'] = [''.join((str(a), str(b))) for a, b in zip(brook['addno1'], brook['addno2'])]
brook.loc[:,'prop_addr'] = [' '.join((str(a), str(b))) for a, b in zip(brook['prop_addr'], brook['addst1'])]
brook.loc[:,'prop_addr'] = brook.prop_addr.str.strip()
# Append 
brook.loc[:,'own'] = [' '.join((str (a), str(b))) for a, b in zip(brook['firstname1'], brook['lastname1'])]
brook.loc[:,'co'] = [' '.join((str(a), str(b))) for a, b in zip(brook['firstname2'], brook['lastname2'])]
brook = brook.replace({' ': None, '': None})
brook = brook.rename(columns = {
    'parcel-id': 'gisid',
    'addst2': 'unit',
    'restotlval': 'ass_val',
    'saleprice': 'sale_p',
    'saledate': 'sale_d'
})
brook = brook.replace({r'^, ': '', r' ,': '', r', nan': '', r'nan': '', r'None, ': '', r', None': ''}, regex=True)
brook = brook.replace({' ': None, '': None})
brook['sale_p'].values[brook['sale_p'].values < 1] = None
brook['year'] = 'FY2020'
brook = brook[['gisid', 'town', 'prop_addr', 'unit', 'own', 'co', 'own_addr', 'ass_val', 'year', 'sale_d', 'sale_p']]

/home/ericrobskyhuntley/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (20,134,136,137,138,139,140,141,143,152,154,155,156,161,165,173) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-11-e3e36685ca37>:33: RuntimeWarning: invalid value encountered in less
  brook['sale_p'].values[brook['sale_p'].values < 1] = None


In [75]:
all_assess = pd.concat([som, med, cam, bos, brook], ignore_index=True)

In [76]:
all_assess.loc[:,'prop_addr'] = all_assess.prop_addr.str.lstrip('0').str.strip()
all_assess.loc[:,'own_addr'] = all_assess.own_addr.str.lstrip('0').str.strip()
all_assess.loc[:,'co'] = all_assess['co'].replace({r'C/O ': '', r'S/O ': '', r'ATTN: ': '', r'ATTN ': ''}, regex=True)
all_assess = all_assess.replace({r'None': '', 'nan': ''}, regex=True)
all_assess = all_assess.replace({' ': None, '': None})
all_assess = all_assess[~pd.isnull(all_assess['gisid'])]
all_assess = all_assess.replace({pd.np.nan: None})

<ipython-input-76-b5e407aef94e>:7: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  all_assess = all_assess.replace({pd.np.nan: None})


In [77]:
def tupleize(row):
    if (row['co'] is not None) & (row['own'] is not None):
        return tuple([row['own'], row['co']])
    elif (row['co'] is None) & (row['own'] is not None):
        return tuple([row['own']])
    else:
        return None
    

all_assess['owners'] = all_assess.apply(tupleize, axis=1)

In [47]:
# Convert to dictionary (expected by Dedupe)
all_assess_dict = all_assess[['owners','own_addr']].to_dict('index')

In [50]:
settings_file = './training/learned_settings'
training_file = './training/training.json'

# If settings exist, read from existing.
if os.path.exists(settings_file):
    print('Reading learned settings from', settings_file)
    with open(settings_file, 'rb') as f:
        deduper = dedupe.StaticDedupe(f)
else:
    # Tell Dedupe which fields are used to identify duplicates.
    fields = [
        {'field': 'owners', 'variable name': 'owners', 'type': 'Set'},
        {'field': 'own_addr', 'variable name': 'own_addr', 'type': 'Address'}
        ]
    deduper = dedupe.Dedupe(fields)
    # If training file exists, read it...
    if os.path.exists(training_file):
        print('reading labeled examples from ', training_file)
        with open(training_file, 'rb') as f:
            deduper.prepare_training(all_assess_dict, f)
    # Otherwise, prepare a training set...
    else:
        deduper.prepare_training(all_assess_dict)
    # Start supervised labeling.
    dedupe.console_label(deduper)
    deduper.train()
    # Write settings and training sets.
    with open(training_file, 'w') as tf:
        deduper.write_training(tf)
    with open(settings_file, 'wb') as sf:
        deduper.write_settings(sf)

INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (sortedAcronym, own_addr), SimplePredicate: (wholeSetPredicate, owners))
owners : ('HISTORIC SOUTH END LP', 'MARVIN F POER & COMPANY')
own_addr : PO BOX 52427, ATLANTA, GA 30355

owners : ('HISTORIC SOUTH END LP',)
own_addr : PO BOX 52427, ATLANTA, GA 30355

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


y


owners : ('PRESIDENT & FELLOWS OF HARVARD COLLEGE', 'HARVARD REAL ESTATE INC.')
own_addr : HOLYOKE CENTER - ROOM #1000, 1350 MASS AVE, CAMBRIDGE, MA 02138

owners : ('PRESIDENT & FELLOWS OF HARVARD COLLEGE', 'HARVARD REAL ESTATE SEVICES')
own_addr : 1350  MASS AVE,HOLYOKE CNTR #1017, CAMBRIDGE, MA 02138

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


y


INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (firstSetElementPredicate, owners), SimplePredicate: (wholeFieldPredicate, own_addr))
owners : ('YOGEL, ERIC B. TRUSTEE OF', 'PEARL # 100-3 TRUST')
own_addr : P.O. BOX 398063, CAMBRIDGE, MA 02139

owners : ('YOGEL, ERIC B. TRUSTEE OF', 'PEARL #96-2 TRUSTE')
own_addr : P.O. BOX 398063, CAMBRIDGE, MA 02139

2/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

owners : ('HARDING SHIRLEY A TRSTS',)
own_addr : PO BOX 51544, BOSTON, MA 02205

owners : ('JAMES DWIGHT',)
own_addr : 590 BLUE HILL AV, DORCHESTER, MA 02121

118/10 positive, 193/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


owners : ('LEE JENNIFER M ',)
own_addr : 7 CENTRAL ST, WINCHESTER, MA 01890

owners : ('WOOD KAREN F',)
own_addr : 7 SARANAC ST, DORCHESTER, MA 02122

118/10 positive, 194/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling
INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 1.000000, score 0.8457269450057222
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (oneGramFingerprint, own_addr), SimplePredicate: (sameThreeCharStartPredicate, own_addr))
INFO:dedupe.training:SimplePredicate: (firstSetElementPredicate, owners)


In [59]:
# Identify clusters based on training dataset.
# Higher threshold is less tolerant of differences between names/addresses.
clustered_dupes = deduper.partition(all_assess_dict, threshold = 0.5)

# How many sets are there?
print('Number of sets', len(clustered_dupes))

INFO:dedupe.blocking:10000, 0.1447202 seconds
INFO:dedupe.blocking:20000, 0.2868192 seconds
INFO:dedupe.blocking:30000, 0.4337112 seconds
INFO:dedupe.blocking:40000, 0.5835662 seconds
INFO:dedupe.blocking:50000, 0.7267232 seconds
INFO:dedupe.blocking:60000, 0.8654432 seconds
INFO:dedupe.blocking:70000, 1.0016342 seconds
INFO:dedupe.blocking:80000, 1.1351392 seconds
INFO:dedupe.blocking:90000, 1.2731762 seconds
INFO:dedupe.blocking:100000, 1.4262552 seconds
INFO:dedupe.blocking:110000, 1.5795912 seconds
INFO:dedupe.blocking:120000, 1.7387642 seconds
INFO:dedupe.blocking:130000, 1.8897642 seconds
INFO:dedupe.blocking:140000, 2.0337372 seconds
INFO:dedupe.blocking:150000, 2.1779522 seconds
INFO:dedupe.blocking:160000, 2.3265102 seconds
INFO:dedupe.blocking:170000, 2.4692312 seconds
INFO:dedupe.blocking:180000, 2.6154152 seconds
INFO:dedupe.blocking:190000, 2.7589062 seconds
INFO:dedupe.blocking:200000, 2.9009222 seconds
INFO:dedupe.blocking:210000, 3.0433492 seconds


Number of sets 172302


In [53]:
# Create empty arrays to hold results.
rid = []
clst = []
conf = []
count = []

# Iterate over results...
for cluster_id, (records, scores) in enumerate(clustered_dupes):
    for record_id, score in zip(records, scores):
        # Append record id
        # Corresponds to index of assessor dataframe.
        rid.append(record_id)
        # Append cluster ID.
        clst.append(cluster_id)
        # Append confidence score.
        conf.append(score)

# Build new dataframe using result arrays.
clust = pd.DataFrame(list(zip(clst, conf)), 
                  columns =['clst', 'conf'],
                  index = rid
                 )

In [78]:
# Join clusters to assessors dataframe.
all_assess = all_assess.join(clust)

In [80]:
# Read spatial data
parcels_gdf = gpd.read_file('./data/parcels/mamas_parcels.shp')
parcels_gdf = parcels_gdf.rename(columns = {
    'pid': 'gisid'
}).drop_duplicates(subset=['gisid', 'town'])
parcels_gdf = parcels_gdf[~pd.isnull(parcels_gdf['gisid'])]
parcels_gdf = parcels_gdf[~pd.isnull(parcels_gdf['geometry'])]
# parcels_gdf.loc[:,'geometry'] = parcels_gdf.geometry.centroid
centroid = parcels_gdf.geometry.centroid
parcels_gdf.loc[:,'lat'] = centroid.y
parcels_gdf.loc[:,'lon'] = centroid.x

<ipython-input-80-7d7fbb14c5ae>:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = parcels_gdf.geometry.centroid


In [81]:
all_assess = parcels_gdf.merge(all_assess, on=['town', 'gisid'], how='right')
all_assess = all_assess[~np.isnan(all_assess.lat)]

In [82]:
# Hard-coding this count logic saves a ton of time for each PostgreSQL query.
all_assess = all_assess.merge(all_assess.groupby('clst').count()[['gisid']].rename(columns={'gisid': 'count'}), on=['clst', 'clst'], how='left')

In [83]:
pg_engine = create_engine(PG_CONNECT)
all_assess.to_postgis("props", con=pg_engine, schema='public', if_exists='replace', index=True, index_label='id')